### Jul AdventKalender D15

https://adventofcode.com/2024/day/15

#### Part 1

In [1]:
import numpy as np

def read_file():
    input_file_path = "data/input15.txt"
    map_boxes = []
    moves = ""
    def process_line(line, if_upper):
        nonlocal moves
        if if_upper:
            map_boxes.append(list(line.strip()))
        else:
            moves += line.strip()

    with open(input_file_path, 'r') as file:
        if_upper = True
        for line in file:
            if not line.strip():
                if_upper = False
                continue
            if if_upper:
                process_line(line, True)
            else:
                process_line(line, False)
            
    return np.array(map_boxes), moves

map_boxes, moves = read_file()
start_pos = tuple(i[0] for i in np.where(map_boxes=="@"))

In [2]:
directions = {
    "<": (0, -1),
    ">": (0, 1),
    "^": (-1, 0),
    "v": (1, 0)
}

def move(pos, direction):
    global map_boxes
    dx, dy = directions[direction]
    nx, ny = pos[0] + dx, pos[1] + dy
    if map_boxes[nx, ny] == "#":
        return pos
    if map_boxes[nx, ny] == ".":
        map_boxes[nx, ny] = "@"
        map_boxes[pos[0], pos[1]] = "."
        return (nx, ny)
    if map_boxes[nx, ny] == "O":
        tmp_boxes = [(nx, ny)]
        current_box = (nx, ny)
        # get all the movable boxes
        while(True):
            nnx, nny = current_box[0] + dx, current_box[1] + dy
            if map_boxes[nnx, nny] == "#":
                return pos
            if map_boxes[nnx, nny] == ".":
                break
            if map_boxes[nnx, nny] == "O":
                current_box = (nnx, nny)
                tmp_boxes.append((nnx, nny))
                continue
            print("what did you encounter here?! ", map_boxes[nnx, nny])
        # move all the boxes
        for box in tmp_boxes:
            map_boxes[box[0]+dx, box[1]+dy] = "O"
        map_boxes[tmp_boxes[0][0], tmp_boxes[0][1]] = "@"
        map_boxes[pos[0], pos[1]] = "."
        return tmp_boxes[0]
    print("what did you encounter here?! ", map_boxes[nx, ny])
    return pos

def sum_coor():
    global map_boxes
    total = 0
    ids_x, ids_y = np.where(map_boxes=="O")
    for id_x in ids_x:
        total += 100*id_x
    for id_y in ids_y:
        total += id_y
    return total

In [3]:
current_pos = start_pos
for i in moves:
    current_pos = move(current_pos, i)
    
sum_coor()

1516281

#### Part 2

In [4]:
import numpy as np

def read_file2():
    input_file_path = "data/input15.txt"
    map_boxes = []
    moves = ""
    def process_line(line, if_upper):
        nonlocal moves
        if if_upper:
            map_boxes.append(list(line.strip().replace("#","##").replace("O","[]").replace(".","..").replace("@","@.")))
        else:
            moves += line.strip()

    with open(input_file_path, 'r') as file:
        if_upper = True
        for line in file:
            if not line.strip():
                if_upper = False
                continue
            if if_upper:
                process_line(line, True)
            else:
                process_line(line, False)
            
    return np.array(map_boxes), moves

map_boxes_expanded, moves = read_file2()
start_pos = tuple(i[0] for i in np.where(map_boxes_expanded=="@"))

In [5]:
directions = {
    "<": (0, -1),
    ">": (0, 1),
    "^": (-1, 0),
    "v": (1, 0)
}

def move(pos, direction):
    global map_boxes_expanded
    dx, dy = directions[direction]
    nx, ny = pos[0] + dx, pos[1] + dy
    if map_boxes_expanded[nx, ny] == "#":
        return pos
    if map_boxes_expanded[nx, ny] == ".":
        map_boxes_expanded[nx, ny] = "@"
        map_boxes_expanded[pos[0], pos[1]] = "."
        return (nx, ny)
    if map_boxes_expanded[nx, ny] in ["[","]"]:
        if direction == "<":
            current_boxes = [[(nx, ny-1),(nx, ny)]]
        else:
            if map_boxes_expanded[nx, ny] == "[": 
                current_boxes = [[(nx, ny),(nx, ny+1)]]
            else: 
                current_boxes = [[(nx, ny-1),(nx, ny)]]
        tmp_boxes = current_boxes
        # get all the movable boxes
        while(True):
            next_boxes = []
            if_clear = [False]*len(current_boxes)
            for i in range(len(current_boxes)):
                current_box = current_boxes[i]
                # up down
                if direction in ["^", "v"]:
                    nnx_left, nny_left = current_box[0][0] + dx, current_box[0][1] + dy
                    nnx_right, nny_right = current_box[1][0] + dx, current_box[1][1] + dy
                    if map_boxes_expanded[nnx_left, nny_left] == "#" or map_boxes_expanded[nnx_right, nny_right] == "#":
                        return pos
                    if map_boxes_expanded[nnx_left, nny_left] == "." and map_boxes_expanded[nnx_right, nny_right] == ".":
                        if_clear[i] = True
                        continue
                    if map_boxes_expanded[nnx_left, nny_left] in ["[","]"] or (map_boxes_expanded[nnx_right, nny_right] in ["[","]"]):
                        if map_boxes_expanded[nnx_left, nny_left] == "[": 
                            next_boxes.append([(nnx_left, nny_left),(nnx_right, nny_right)])
                        elif map_boxes_expanded[nnx_left, nny_left] == "]": 
                            next_boxes.append([(nnx_left, nny_left-1),(nnx_left, nny_left)])
                        if map_boxes_expanded[nnx_right, nny_right] == "[": 
                            next_boxes.append([(nnx_right, nny_right),(nnx_right, nny_right+1)])
                # left right
                else:
                    if direction == "<":
                        nnx, nny = current_box[0][0] + dx, current_box[0][1] + dy
                    else:
                        nnx, nny = current_box[1][0] + dx, current_box[1][1] + dy
                    if map_boxes_expanded[nnx, nny] == "#":
                        return pos
                    if map_boxes_expanded[nnx, nny] == ".":
                        if_clear[i] = True
                        continue
                    if map_boxes_expanded[nnx, nny] == "[":
                        next_boxes.append([(nnx, nny),(nnx + dx, nny + dy)])
                    elif map_boxes_expanded[nnx, nny] == "]":
                        next_boxes.append([(nnx + dx, nny + dy),(nnx, nny)])
                    
            tmp_boxes.extend(next_boxes)
            if all(if_clear):
                break
            current_boxes = next_boxes
        # move all the boxes
        for box in tmp_boxes:
            map_boxes_expanded[box[0][0], box[0][1]] = "."
            map_boxes_expanded[box[1][0], box[1][1]] = "."
        for box in tmp_boxes:
            map_boxes_expanded[box[0][0]+dx, box[0][1]+dy] = "["
            map_boxes_expanded[box[1][0]+dx, box[1][1]+dy] = "]"

        map_boxes_expanded[nx, ny] = "@"
        map_boxes_expanded[pos[0], pos[1]] = "."
        return (nx, ny)
    print("what did you encounter here?! ", map_boxes_expanded[nx, ny])
    return pos

def sum_coor():
    global map_boxes_expanded
    total = 0
    ids_x, ids_y = np.where(map_boxes_expanded=="[")
    for id_x in ids_x:
        total += 100*id_x
    for id_y in ids_y:
        total += id_y
    return total

In [6]:
current_pos = start_pos
for i in moves:
    current_pos = move(current_pos, i)
#     print(map_boxes_expanded)
sum_coor()

1527969